In [2]:
import numpy as np
import pandas as pd
from myfunctions import BPstar
from myfunctions import BPstarmodel
import matplotlib.pyplot as plt
import os
from est_model import estimate_model

# Initialisation, simulation, et replications

works well:
#initialisations des parametres du modele pour simulation
A = np.array([[0.2, 0],[0, 0.4]])
B = np.array([[0.5, 0],[0, 0.3]])
omeg = np.array([0.3, 0.5])
delt = 0.7
n = 500 #longeure de la trajectoire

## Simulation d'une trajectoire

### Paramètres de la simulation:

#### Graphes


## Replications


In [120]:
parameters = {
    'M1': {
        'A': np.array([[0.3, 0.2], [0.1, 0.3]]),
        'B': np.array([[0.1, 0.2], [0.3, 0.2]]),
        'omeg': np.array([0.5, 0.3]),
        'delt': 0.4
        
    },
    'M2': {
        'A': np.array([[0.2, 0], [0, 0.4]]),
        'B': np.array([[0.5, 0], [0, 0.3]]),
        'omeg': np.array([0.3, 0.5]),
        'delt': 0.7
    },
    'M3': {
        'A': np.array([[0.2, 0.0], [0.0, 0.4]]),
        'B': np.array([[0.5, 0.1], [0.3, 0.2]]),
        'omeg': np.array([0.3, 0.5]),
        'delt': 0.7
    },
    'M4': {
        'A': np.array([[0.2, 0.3], [0.4, 0.2]]),
        'B': np.array([[0.2, 0], [0, 0.3]]),
        'omeg': np.array([0.5, 0.3]),
        'delt': -0.5
    }
}

In [94]:
for model_name in parameters:
    parameters[model_name]['real_params'] = [
        parameters[model_name]['omeg'][0],
        *parameters[model_name]['A'][0],
        *parameters[model_name]['B'][0],
        parameters[model_name]['omeg'][1],
        *parameters[model_name]['A'][1],
        *parameters[model_name]['B'][1]
    ] 

print(parameters)

{'M1': {'A': array([[0.3, 0.2],
       [0.1, 0.3]]), 'B': array([[0.1, 0.2],
       [0.3, 0.2]]), 'omeg': array([0.5, 0.3]), 'delt': 0.4, 'real_params': [0.5, 0.3, 0.2, 0.1, 0.2, 0.3, 0.1, 0.3, 0.3, 0.2]}, 'M2': {'A': array([[0.2, 0. ],
       [0. , 0.4]]), 'B': array([[0.5, 0. ],
       [0. , 0.3]]), 'omeg': array([0.3, 0.5]), 'delt': 0.7, 'real_params': [0.3, 0.2, 0.0, 0.5, 0.0, 0.5, 0.0, 0.4, 0.0, 0.3]}, 'M3': {'A': array([[0.2, 0. ],
       [0. , 0.4]]), 'B': array([[0.5, 0.1],
       [0.3, 0.2]]), 'omeg': array([0.3, 0.5]), 'delt': 0.7, 'real_params': [0.3, 0.2, 0.0, 0.5, 0.1, 0.5, 0.0, 0.4, 0.3, 0.2]}, 'M4': {'A': array([[0.2, 0.3],
       [0.4, 0.2]]), 'B': array([[0.2, 0. ],
       [0.3, 0. ]]), 'omeg': array([0.5, 0.3]), 'delt': -0.5, 'real_params': [0.5, 0.2, 0.3, 0.2, 0.0, 0.3, 0.4, 0.2, 0.3, 0.0]}}


In [104]:
model_specs = {
    'M1': {'indices': range(10)},
    'M2': {'indices': [0, 1, 3, 5, 7, 9]},
    'M3': {'indices': [0, 1, 3, 4, 5, 7, 8, 9]},
    'M4': {'indices': [0, 1, 2, 3, 5, 6, 7, 9]}
}

In [119]:
model = 'M2'
es = np.array(parameters[model]['real_params'])
print(f"es: {es}")

cures = np.array([parameters[model]['real_params'][i] for i in model_specs[model]['indices']])
print(f"cures : {cures}")
range(10)

es: [0.3 0.2 0.  0.5 0.  0.5 0.  0.4 0.  0.3]
cures : [0.3 0.2 0.5 0.5 0.4 0.3]


range(0, 10)

### Simulation repliquée

In [5]:
#nombre de replications
nrep = 200

In [6]:
#on va faire des replications pour des tailles de series differentes
sizes = [200, 500, 1000]

In [11]:
#on cree un sous fichier pour y sauvegarder les replications
base_folder = "replications"
os.makedirs(base_folder, exist_ok=True)

### Estimation des replications

In [12]:
RepEst = np.zeros((10, nrep))
np.shape(RepEst)

(10, 200)

In [14]:
#Estimation
base_folder = "replications"
for model in parameters: #on va parcourir les modeles
    model_folder = os.path.join(base_folder, model)        
    print(model_folder)
    for size in sizes: #on regarde les tailles des series
        size_folder = os.path.join(model_folder, f"size_{size}")
        print(size_folder)
        RepEst = np.zeros((10,nrep))
        for i in range(1,nrep+1):
            filename = f"databp_{str(i).zfill(3)}.csv"
            filepath = os.path.join(size_folder, filename)
            if os.path.exists(filepath):
                data = np.loadtxt(filepath, delimiter=',')
                print(f"loaded model: {model}, size: {size}, replication: {filename} with shape {data.shape}")
                omega, A, B = estimate_model(data, 3)
                print(f"model estimated from data {model} {size} :{str(i).zfill(3)}")
                RepEst[0,i-1], RepEst[5,i-1] = omega[0], omega[1]
                RepEst[1,i-1], RepEst[2,i-1] = A[0,0], A[0,1]
                RepEst[6,i-1], RepEst[7,i-1] = A[1,0], A[1,1]
                RepEst[3,i-1], RepEst[4,i-1] = B[0,0], B[0,1]
                RepEst[8,i-1], RepEst[9,i-1] = B[1,0], B[1,1]
                estimname = f"repestim_{model}_{size}.csv"
                estimpath = os.path.join(size_folder, estimname)
                np.savetxt(estimpath, RepEst, delimiter=",")
                print(f"estimation saved to {estimpath}")
                
            else:
                print(f"File {filename} does not exist.")


replications\M1
replications\M1\size_200
loaded model: M1, size: 200, replication: databp_001.csv with shape (200, 2)
model estimated from data M1 200 :001
estimation saved to replications\M1\size_200\repestim_M1_200.csv
loaded model: M1, size: 200, replication: databp_002.csv with shape (200, 2)
model estimated from data M1 200 :002
estimation saved to replications\M1\size_200\repestim_M1_200.csv
loaded model: M1, size: 200, replication: databp_003.csv with shape (200, 2)
model estimated from data M1 200 :003
estimation saved to replications\M1\size_200\repestim_M1_200.csv
loaded model: M1, size: 200, replication: databp_004.csv with shape (200, 2)
model estimated from data M1 200 :004
estimation saved to replications\M1\size_200\repestim_M1_200.csv
loaded model: M1, size: 200, replication: databp_005.csv with shape (200, 2)
model estimated from data M1 200 :005
estimation saved to replications\M1\size_200\repestim_M1_200.csv
loaded model: M1, size: 200, replication: databp_006.csv wi

### Statistiques sur les estimations


generation d'un fichier csv modelestim_{model}_{size}.csv qui a la bonne taille de parametres 

In [116]:
for model in parameters:
    for size in sizes:
        model_folder = os.path.join(base_folder, model)
        size_folder = os.path.join(model_folder, f"size_{size}")
        filename = f"repestim_{model}_{size}.csv"
        filepath = os.path.join(size_folder, filename)

        repestim = np.loadtxt(filepath, delimiter=',')
        print(f"loaded data: {filepath} with shape {np.shape(repestim)}")
        #on defini les vecteurs d'estimation specifiques aux modeles avec les parametres d'interet seulement
        
        current_estim = np.array([repestim[i] for i in model_specs[model]['indices']])
        meanest = current_estim.mean(axis=1)
        stdest = current_estim.std(axis=1)
        minest = current_estim.min(axis=1)
        maxest = current_estim.max(axis=1)
        medianest = np.median(current_estim, axis=1)


        festimname = f"modelestim_{model}_{size}.csv"
        festimpath = os.path.join(size_folder,festimname)
        np.savetxt(festimpath, current_estim, delimiter=",")
        print(f"saved final size estimation {festimname}")
        
        

loaded data: replications\M1\size_200\repestim_M1_200.csv with shape (10, 200)
saved final size estimationmodelestim_M1_200.csv
loaded data: replications\M1\size_500\repestim_M1_500.csv with shape (10, 200)
saved final size estimationmodelestim_M1_500.csv
loaded data: replications\M1\size_1000\repestim_M1_1000.csv with shape (10, 200)
saved final size estimationmodelestim_M1_1000.csv
loaded data: replications\M2\size_200\repestim_M2_200.csv with shape (10, 200)
saved final size estimationmodelestim_M2_200.csv
loaded data: replications\M2\size_500\repestim_M2_500.csv with shape (10, 200)
saved final size estimationmodelestim_M2_500.csv
loaded data: replications\M2\size_1000\repestim_M2_1000.csv with shape (10, 200)
saved final size estimationmodelestim_M2_1000.csv
loaded data: replications\M3\size_200\repestim_M3_200.csv with shape (10, 200)
saved final size estimationmodelestim_M3_200.csv
loaded data: replications\M3\size_500\repestim_M3_500.csv with shape (10, 200)
saved final size es

In [80]:
for model in parameters:
    for size in sizes:
        model_folder = os.path.join(base_folder, model)
        size_folder = os.path.join(model_folder, f"size_{size}")
        filename = f"repestim_{model}_{size}.csv"
        filepath = os.path.join(size_folder, filename)

        repestim = np.loadtxt(filepath, delimiter=',')
        print(f"loaded data: {filepath} with shape {np.shape(repestim)}")
        #on defini les vecteurs d'estimation specifiques aux modeles avec les parametres d'interet seulement
        

        #M1 prend tout pas besoin
        if model == 'M1':
            print(model)
            meanest = repestim.mean(axis=1)
            stdest = repestim.std(axis=1)
            minest = repestim.min(axis=1)
            maxest = repestim.max(axis=1)
            medianest = np.median(repestim, axis=1)
            num_params = 10
            real_params = np.array([0.5, 0.3, 0.2, 0.1, 0.2, 0.3, 0.1, 0.3, 0.3, 0.2])
            festimname = f"modelestim_{model}_{size}.csv"
            festimpath = os.path.join(size_folder,festimname)
            np.savetxt(festimpath, repestim, delimiter=",")


        if model == 'M2':
            #M2 A et B diag
            print(model)
            repestimM2 = np.zeros((6,nrep))
            repestimM2[0,:] = repestim[0,:] #omega1
            repestimM2[1,:] = repestim[1,:] #alpha11
            repestimM2[2,:] = repestim[3,:] #beta11
            repestimM2[3,:] = repestim[5,:] #omega2
            repestimM2[4,:] = repestim[7,:] #alpha22
            repestimM2[5,:] = repestim[9,:] #beta22

            meanest = repestimM2.mean(axis=1)
            stdest = repestimM2.std(axis=1)
            minest = repestimM2.min(axis=1)
            maxest = repestimM2.max(axis=1)
            medianest = np.median(repestimM2, axis=1)
            num_params = 6
            real_params = np.array([0.3, 0.2, 0.5, 0.5, 0.4, 0.3])
            
            festimname = f"modelestim_{model}_{size}.csv"
            festimpath = os.path.join(size_folder,festimname)
            np.savetxt(festimpath, repestimM2, delimiter=",")

        if model == 'M3':
            #M3 A diag B non-diag
            print(model)
            repestimM3 = np.zeros((8,nrep))
            repestimM3[0,:] = repestim[0,:] #omega1
            repestimM3[1,:] = repestim[1,:] #alpha11
            repestimM3[2,:] = repestim[3,:] #beta11
            repestimM3[3,:] = repestim[4,:] #beta12
            repestimM3[4,:] = repestim[5,:] #omega2
            repestimM3[5,:] = repestim[7,:] #alpha22
            repestimM3[6,:] = repestim[8,:] #beta21
            repestimM3[7,:] = repestim[9,:] #beta22
                
            meanest = repestimM3.mean(axis=1)
            stdest = repestimM3.std(axis=1)
            minest = repestimM3.min(axis=1)
            maxest = repestimM3.max(axis=1)
            medianest = np.median(repestimM3, axis=1)
            num_params = 8
            real_params = np.array([0.3, 0.2, 0.5, 0.1, 0.5, 0.4, 0.3, 0.2])
            
            festimname = f"modelestim_{model}_{size}.csv"
            festimpath = os.path.join(size_folder,festimname)
            np.savetxt(festimpath, repestimM3, delimiter=",")

        if model == 'M4':
            #M4 A non diag B diag
            print(model)
            repestimM4 = np.zeros((8,nrep))
            repestimM4[0,:] = repestim[0,:] #omega1
            repestimM4[1,:] = repestim[1,:] #alpha11
            repestimM4[2,:] = repestim[2,:] #alpha12
            repestimM4[3,:] = repestim[3,:] #beta11
            repestimM4[4,:] = repestim[5,:] #omega2
            repestimM4[5,:] = repestim[6,:] #alpha21
            repestimM4[6,:] = repestim[7,:] #alpha22
            repestimM4[7,:] = repestim[9,:] #beta22
                
            meanest = repestimM4.mean(axis=1)
            stdest = repestimM4.std(axis=1)
            minest = repestimM4.min(axis=1)
            maxest = repestimM4.max(axis=1)
            medianest = np.median(repestimM4, axis=1)
            num_params = 8
            real_params = np.array([0.5, 0.2, 0.3, 0.2, 0.3, 0.4, 0.2, 0.3])
            
            relname = f"real_param_{model}"
            festimname = f"modelestim_{model}_{size}.csv"
            festimpath = os.path.join(size_folder,festimname)
            np.savetxt(festimpath, repestimM4, delimiter=",")
            print(f"saved estimation vector {festimname}")

        

loaded data: replications\M1\size_200\repestim_M1_200.csv with shape (10, 200)
M1
loaded data: replications\M1\size_500\repestim_M1_500.csv with shape (10, 200)
M1
loaded data: replications\M1\size_1000\repestim_M1_1000.csv with shape (10, 200)
M1
loaded data: replications\M2\size_200\repestim_M2_200.csv with shape (10, 200)
M2
loaded data: replications\M2\size_500\repestim_M2_500.csv with shape (10, 200)
M2
loaded data: replications\M2\size_1000\repestim_M2_1000.csv with shape (10, 200)
M2
loaded data: replications\M3\size_200\repestim_M3_200.csv with shape (10, 200)
M3
loaded data: replications\M3\size_500\repestim_M3_500.csv with shape (10, 200)
M3
loaded data: replications\M3\size_1000\repestim_M3_1000.csv with shape (10, 200)
M3
loaded data: replications\M4\size_200\repestim_M4_200.csv with shape (10, 200)
M4
loaded data: replications\M4\size_500\repestim_M4_500.csv with shape (10, 200)
M4
loaded data: replications\M4\size_1000\repestim_M4_1000.csv with shape (10, 200)
M4


#### just pour savoir ou on est et utiliser un modele en particulier

selection de l'estimation

In [77]:
base_folder = "replications"

model = 'M1'
size = 200

model_folder = os.path.join(base_folder, model)
size_folder = os.path.join(model_folder, f"size_{size}")
filename = f"repestim_{model}_{size}.csv"
filepath = os.path.join(size_folder, filename)

repestim = np.loadtxt(filepath, delimiter=',')
print(f"loaded data: {filepath} with shape {np.shape(repestim)}")

loaded data: replications\M1\size_200\repestim_M1_200.csv with shape (10, 200)


In [79]:
#on defini les vecteurs d'estimation specifiques aux modeles avec les parametres d'interet seulement
#M1 prend tout pas besoin
if model == 'M1':
    print(model)
    meanest = repestim.mean(axis=1)
    stdest = repestim.std(axis=1)
    minest = repestim.min(axis=1)
    maxest = repestim.max(axis=1)
    medianest = np.median(repestim, axis=1)
    num_params = 10
    real_params = np.array([0.5, 0.3, 0.2, 0.1, 0.2, 0.3, 0.1, 0.3, 0.3, 0.2])



if model == 'M2':
    #M2 A et B diag
    print(model)
    repestimM2 = np.zeros((6,nrep))
    repestimM2[0,:] = repestim[0,:] #omega1
    repestimM2[1,:] = repestim[1,:] #alpha11
    repestimM2[2,:] = repestim[3,:] #beta11
    repestimM2[3,:] = repestim[5,:] #omega2
    repestimM2[4,:] = repestim[7,:] #alpha22
    repestimM2[5,:] = repestim[9,:] #beta22

    meanest = repestimM2.mean(axis=1)
    stdest = repestimM2.std(axis=1)
    minest = repestimM2.min(axis=1)
    maxest = repestimM2.max(axis=1)
    medianest = np.median(repestimM2, axis=1)
    num_params = 6
    real_params = np.array([0.3, 0.2, 0.5, 0.5, 0.4, 0.3])
    

if model == 'M3':
    #M3 A diag B non-diag
    print(model)
    repestimM3 = np.zeros((8,nrep))
    repestimM3[0,:] = repestim[0,:] #omega1
    repestimM3[1,:] = repestim[1,:] #alpha11
    repestimM3[2,:] = repestim[3,:] #beta11
    repestimM3[3,:] = repestim[4,:] #beta12
    repestimM3[4,:] = repestim[5,:] #omega2
    repestimM3[5,:] = repestim[7,:] #alpha22
    repestimM3[6,:] = repestim[8,:] #beta21
    repestimM3[7,:] = repestim[9,:] #beta22
        
    meanest = repestimM3.mean(axis=1)
    stdest = repestimM3.std(axis=1)
    minest = repestimM3.min(axis=1)
    maxest = repestimM3.max(axis=1)
    medianest = np.median(repestimM3, axis=1)
    num_params = 8
    real_params = np.array([0.3, 0.2, 0.5, 0.1, 0.5, 0.4, 0.3, 0.2])
    
if model == 'M4':
    #M4 A non diag B diag
    print(model)
    repestimM4 = np.zeros((8,nrep))
    repestimM4[0,:] = repestim[0,:] #omega1
    repestimM4[1,:] = repestim[1,:] #alpha11
    repestimM4[2,:] = repestim[2,:] #alpha12
    repestimM4[3,:] = repestim[3,:] #beta11
    repestimM4[4,:] = repestim[5,:] #omega2
    repestimM4[5,:] = repestim[6,:] #alpha21
    repestimM4[6,:] = repestim[7,:] #alpha22
    repestimM4[7,:] = repestim[9,:] #beta22
        
    meanest = repestimM4.mean(axis=1)
    stdest = repestimM4.std(axis=1)
    minest = repestimM4.min(axis=1)
    maxest = repestimM4.max(axis=1)
    medianest = np.median(repestimM4, axis=1)
    num_params = 8
    real_params = np.array([0.5, 0.2, 0.3, 0.2, 0.3, 0.4, 0.2, 0.3])
    


###COMMENT IMPORTER DU DICTIONNAIRE

M1
(10, 200)


In [72]:
# Print the computed statistics
print(f"Mean: {meanest}")
print(f"Standard Deviation: {stdest}")
print(f"Min: {minest}")
print(f"Max: {maxest}")
print(f"Median: {medianest}")


Mean: [0.66999403 0.18711713 0.25021293 0.09180654 0.18342664 0.41193972
 0.13248812 0.22999618 0.32627911 0.20088302]
Standard Deviation: [0.42870456 0.37576206 0.32134247 0.07207014 0.56213784 0.39825706
 0.38854974 0.30657464 0.55114944 0.07685687]
Min: [-0.42556396 -0.88849385 -0.56043558 -0.13513766 -1.82483914 -0.77865844
 -1.2774045  -0.79670788 -1.79452202 -0.0343877 ]
Max: [2.55003187 1.32235286 1.44754039 0.25942235 2.33197136 2.33096373
 1.46336272 1.1648422  2.26391365 0.41248902]
Median: [0.59213215 0.20088835 0.2320105  0.09364528 0.17842183 0.41375135
 0.13378036 0.23851811 0.35277731 0.19916399]


In [73]:
print(real_params)
len(real_params)
print(np.shape(real_params))

[0.5 0.3 0.2 0.1 0.2 0.3 0.1 0.3 0.3 0.2]
(10,)


In [76]:
MADE = np.zeros((num_params))
print(MADE)
print(np.shape(MADE))
for i in range (0, nrep):
    MADE += abs(mestim[:,i] - real_params)*(1/nrep)
print(MADE)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
(10,)
[0.32201392 0.30236081 0.25311604 0.05891005 0.42772159 0.3057536
 0.29185527 0.24012802 0.42264402 0.05961815]


####

In [21]:
for model in parameters:
    for size in sizes:
        print(model, size)
        MADE = sum()

M1 200
M1 500
M1 1000
M2 200
M2 500
M2 1000
M3 200
M3 500
M3 1000
M4 200
M4 500
M4 1000


In [ ]:
# Create a figure with subplots
fig, axs = plt.subplots(num_params, 1, figsize=(8, 2 * num_params), sharex=True)
# Loop through each parameter and create a box plot
for i in range(num_params):
    axs[i].boxplot(mestim[i, :], vert=False, patch_artist=True)
    axs[i].axvline(x=real_params[i], color='r', linestyle='-', linewidth=1, label='Valeur réelle')
    axs[i].set_title(f'Paramètre {i + 1}')
    axs[i].legend()

fig.suptitle(f"{model} (n = {size})", fontsize=16)

plt.xlabel('Valeurs')
plt.tight_layout(rect=[0, 0, 1, 0.96])  # Adjust layout to make room for the title
plt.show()

In [31]:
summary_stats = {}
for model in parameters:
    # Create an empty list to store data for each size
    size_data = []
    for size in sizes:
        model_folder = os.path.join(base_folder, model)
        size_folder = os.path.join(model_folder, f"size_{size}")
        filename = f"repestim_{model}_{size}.csv"
        filepath = os.path.join(size_folder, filename)

        repestim = np.loadtxt(filepath, delimiter=',')

        # Apply the selection logic based on the model
        if model == 'M1':
            repestim_subset = repestim 
        elif model == 'M2':
            repestim_subset = repestim[[0, 1, 3, 5, 7, 9], :]
        elif model == 'M3':
            repestim_subset = repestim[[0, 1, 3, 4, 5, 7, 8, 9], :]
        elif model == 'M4':
            repestim_subset = repestim[[0, 1, 2, 3, 5, 6, 7, 9], :]
        else:
            print("model does not exist")
            continue # Skip to the next model

        # Calculate statistics for the subset
        meanest = repestim_subset.mean(axis=1)
        stdest = repestim_subset.std(axis=1)
        medianest = np.median(repestim_subset, axis=1)
        minest = repestim_subset.min(axis=1) 
        maxest = repestim_subset.max(axis=1)

        # Append the results to size_data
        size_data.append(pd.DataFrame({
            'Size': [size] * len(meanest),
            'Parameter': np.arange(1, len(meanest) + 1),  
            'Mean': meanest,
            'Std': stdest,
            'Median': medianest,
            'Min': minest,      
            'Max': maxest        
        }))

    # Concatenate the data for all sizes into a single DataFrame
    summary_stats[model] = pd.concat(size_data, ignore_index=True)

# --- Display or Save the DataFrames ---

for model, df in summary_stats.items():
    print(f"----- Summary Statistics for Model {model} -----")
    print(df)
    print("\n")

    # Save each DataFrame to a CSV file
    df.to_csv(f"{model}_summary_stats.csv", index=False)
 

----- Summary Statistics for Model M1 -----
    Size  Parameter      Mean       Std    Median       Min       Max
0    200          1  0.669994  0.428705  0.592132 -0.425564  2.550032
1    200          2  0.187117  0.375762  0.200888 -0.888494  1.322353
2    200          3  0.250213  0.321342  0.232011 -0.560436  1.447540
3    200          4  0.091807  0.072070  0.093645 -0.135138  0.259422
4    200          5  0.183427  0.562138  0.178422 -1.824839  2.331971
5    200          6  0.411940  0.398257  0.413751 -0.778658  2.330964
6    200          7  0.132488  0.388550  0.133780 -1.277405  1.463363
7    200          8  0.229996  0.306575  0.238518 -0.796708  1.164842
8    200          9  0.326279  0.551149  0.352777 -1.794522  2.263914
9    200         10  0.200883  0.076857  0.199164 -0.034388  0.412489
10   500          1  0.601877  0.258789  0.566845  0.042159  1.832934
11   500          2  0.216483  0.249137  0.258764 -0.650872  0.851143
12   500          3  0.248830  0.205536  0.223